## Paso 4 – Transfer Learning con MobileNetV2

En este notebook se desarrolla el punto 4 del caso práctico:

**"Modelo basado en transfer learning"**

En esta sección se implementa un modelo de clasificación mediante *transfer learning*, reutilizando una red neuronal convolucional preentrenada (**MobileNetV2**) y añadiendo una nueva "cabeza" personalizada para adaptarla a nuestro problema de clasificación de 6 clases.

### Características del modelo: detalle de cada capa añadida

- **`base_model`**  
  Modelo base `MobileNetV2`, previamente entrenado sobre ImageNet. Se usa como extractor de características y está congelado (`trainable = False`), por lo que sus pesos no se actualizan durante el entrenamiento.

- **`GlobalAveragePooling2D()`**  
  Sustituye a `Flatten()` como capa de transición. En lugar de aplanar directamente las salidas del modelo base, realiza un promedio sobre cada mapa de activación. Esto reduce el número de parámetros y mejora la generalización del modelo.

- **`Dense(128, activation='relu')`**  
  Capa totalmente conectada con 128 neuronas. Utiliza la función de activación ReLU para introducir no linealidad y permitir el aprendizaje de combinaciones más abstractas de las características extraídas por el modelo base.

- **`Dropout(0.5)`**  
  Durante el entrenamiento, desactiva aleatoriamente el 50% de las neuronas de la capa anterior. Esto reduce la dependencia de ciertas neuronas y ayuda a evitar el sobreajuste (*overfitting*).

- **`Dense(6, activation='softmax')`**  
  Capa de salida con 6 neuronas, una por cada clase del problema de clasificación. La activación softmax convierte las salidas en probabilidades, lo que permite asignar una clase final a cada imagen.

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.callbacks import EarlyStopping

from utils.dataloader import load_data_npy, PreprocessedDataGenerator
from utils.model_utils import save_model_and_history, save_test_results

2025-05-31 07:46:45.600959: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-31 07:46:45.619319: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-31 07:46:45.659726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748677605.718763     343 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748677605.735634     343 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748677605.783823     343 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
# --- Carga de datos desde .npy ---
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

print("Cargadas imágenes y categorias")

# --- Generadores ---
def preprocess_fn(x):
    return mobilenet_preprocess(x)  # o vgg_preprocess(x)

# Generadores adaptados
train_gen = PreprocessedDataGenerator(images_train, categories_train, preprocess_fn=preprocess_fn, resize_to=(224, 224))
val_gen = PreprocessedDataGenerator(images_val, categories_val, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224))
test_gen = PreprocessedDataGenerator(images_test, categories_test, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224))

print("Dividido conjunto completo para entreno, validación y test")

Cargadas imágenes y categorias
Dividido conjunto completo para entreno, validación y test


In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar capas convolucionales del modelo base
base_model.trainable = False  # para feature extraction

# --- Construir modelo con cabeza personalizada ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- Entrenamiento ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=30,
                    callbacks=[early_stop])

# --- Evaluación ---
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}   |  Test Loss: {test_loss:.4f}")

# --- Guardado ---
save_model_and_history(model, history, model_path='transfer_model')
save_test_results('model_extended_cnn', test_loss, test_acc)

2025-05-27 09:44:07.274567: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 296s 897ms/step - accuracy: 0.1895 - loss: 1.8419 - val_accuracy: 0.2454 - val_loss: 1.7337
Epoch 2/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 291s 910ms/step - accuracy: 0.2154 - loss: 1.7269 - val_accuracy: 0.3057 - val_loss: 1.6390
Epoch 3/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 305s 955ms/step - accuracy: 0.2358 - loss: 1.6767 - val_accuracy: 0.2892 - val_loss: 1.6108
Epoch 4/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 340s 1s/step - accuracy: 0.2218 - loss: 1.6618 - val_accuracy: 0.3174 - val_loss: 1.5741
Epoch 5/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 311s 973ms/step - accuracy: 0.2372 - loss: 1.6452 - val_accuracy: 0.3076 - val_loss: 1.5409
Epoch 6/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 340s 1s/step - accuracy: 0.2463 - loss: 1.6340 - val_accuracy: 0.3299 - val_loss: 1.5430
Epoch 7/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 351s 1s/step - accuracy: 0.2420 - loss: 1.6231 - val_accuracy: 0.3284 - val_loss: 1.4932
Epoch 8/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.2432 - loss: 1.619


Test Accuracy: 0.3660
Modelo guardado en: /opt/notebooks/M9/models/transfer_model.h5
Historial guardado en: /opt/notebooks/M9/models/transfer_model_history.json
